# How to Leverage Esri Shapefiles in Snowflake
This project uses geopandas and the Snowflake Python connector to transform and upload a polygon shapefile in order to spatially join data in Snowflake. It uses a polygon shapefile of California Legislative districts and a Snowflake table of cellphone towers, which are spatially joined to understand any potential gaps in access to telecommunication services across the state.

## Prepare Shapefile in Python
### Bring in the necessary packages

In [1]:
import geopandas
import pandas as pd
import json

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

### Open the shapefile in geopandas

In [43]:
shapefile=geopandas.read_file('CA_Legislative_Districts.zip')

In [44]:
shapefile.head()

,FID,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,CVAP_19,HSP_CVAP_1,...,F_CVAP_19,F_HSP_CVAP,F_DOJ_NH_B,F_DOJ_NH_A,F_NH_WHT_C,DISTRICT_N,DISTRICT_L,Shape__Are,Shape__Len,geometry
0,1,1,27242.802700,1,1,None,None,506402,385673,31559,...,0.761595,0.081828,0.011118,0.021122,0.846782,1,1|2.5%,1.229428e+11,2.726995e+06,"POLYGON ((-13375251.157 5160175.897, -13375169..."
1,2,2,116.271873,10,1,None,None,518416,331234,67026,...,0.638935,0.202352,0.160285,0.268605,0.323167,10,10|4.93%,4.922678e+08,1.431753e+05,"POLYGON ((-13513156.271 4655945.846, -13513165..."
2,3,3,1064.780880,11,1,None,None,514921,348085,67399,...,0.675997,0.193628,0.146349,0.151989,0.474855,11,11|4.23%,4.479272e+09,4.545407e+05,"POLYGON ((-13543999.336 4562283.039, -13544052..."
3,4,4,1236.346310,12,1,None,None,482778,340887,41272,...,0.706095,0.121072,0.022506,0.061586,0.779537,12,12|-2.28%,5.187115e+09,3.982014e+05,"POLYGON ((-13649138.606 4553356.640, -13650675..."
4,5,5,489.503387,13,1,None,None,489925,282482,93901,...,0.576582,0.332414,0.117161,0.201100,0.320066,13,13|-0.83%,2.036156e+09,3.559822e+05,"POLYGON ((-13531626.502 4516055.256, -13531626..."


### Drop unneeded columns (optional)
We only retain a few columns that we need for analysis. The geometry column contains the geographic coordinates of the polygons

In [45]:
shapefile=shapefile[['ID','AREA','DISTRICT','POPULATION','geometry']]
shapefile.head()

,ID,AREA,DISTRICT,POPULATION,geometry
0,1,27242.802700,1,506402,"POLYGON ((-13375251.157 5160175.897, -13375169..."
1,2,116.271873,10,518416,"POLYGON ((-13513156.271 4655945.846, -13513165..."
2,3,1064.780880,11,514921,"POLYGON ((-13543999.336 4562283.039, -13544052..."
3,4,1236.346310,12,482778,"POLYGON ((-13649138.606 4553356.640, -13650675..."
4,5,489.503387,13,489925,"POLYGON ((-13531626.502 4516055.256, -13531626..."


### Change Projection
Snowflake geography data type expects the coordinates to be in EPSG:4326, which corresponds to the WGS84 spherical (unprojected) coordinate system. For a spatial join to work correctly, both sets of coordinates must use the same projection. Snowflake geometry data type enables you to use any projection, but changing projecttions in Snowflake is not yet supported. Since the cell towers table uses the geography data type to store the towers' coordinates, we use geopandas to change the projection of the shapefile data to match that of the cell towers.

#### Check the projection
The shapefile is projected with EPSG 3857

In [46]:
shapefile.crs

<Derived Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

#### Reproject the data
Reproject to EPSG 4326

In [48]:
shapefile_wgs84=shapefile.to_crs(4326)
shapefile_wgs84.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Convert geopandas dataframe to a pandas dataframe
In order to load a dataframe into Snowflake via the Snowflake Python connector, it needs to be converted from a geopandas dataframe into a pandas dataframe.

In [49]:
df = pd.DataFrame(shapefile_wgs84)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ID          80 non-null     int64   
 1   AREA        80 non-null     float64 
 2   DISTRICT    80 non-null     object  
 3   POPULATION  80 non-null     int64   
 4   geometry    80 non-null     geometry
dtypes: float64(1), geometry(1), int64(2), object(1)
memory usage: 3.2+ KB


### Convert geometry to string
From my testing, the pandas geometry data type needs to be converted to a string and upcased. I am also renaming it to avoid ambiguity. 

In [50]:
df=df.astype({'geometry':'str'}).rename(columns={'geometry':'POLYGON_COORDINATES'})

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   80 non-null     int64  
 1   AREA                 80 non-null     float64
 2   DISTRICT             80 non-null     object 
 3   POPULATION           80 non-null     int64  
 4   POLYGON_COORDINATES  80 non-null     object 
dtypes: float64(1), int64(2), object(2)
memory usage: 3.2+ KB


## Connect to Snowflake

In [6]:
with open('my_account_auth.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']
    account = data['account']

In [7]:
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    )

### Setup Your Context and Create a Temporary Table to Store the Shapefile Data

In [8]:
conn.cursor().execute("USE WAREHOUSE ADHOC")
conn.cursor().execute("USE DATABASE GEO")
conn.cursor().execute("USE DATABASE GEO")
conn.cursor().execute("CREATE OR REPLACE SCHEMA GEO_BOUNDARIES")
conn.cursor().execute("USE SCHEMA GEO_BOUNDARIES")

In [56]:
conn.cursor().execute("CREATE OR REPLACE TEMPORARY TABLE CA_LEGISLATIVE_DISTRICTS_TMP\
                       (ID number, AREA float, DISTRICT varchar, \
                        POPULATION number, POLYGON_COORDINATES varchar)").fetchone()


('Table CA_LEGISLATIVE_DISTRICTS_TMP successfully created.',)

### Upload Shapefile Data into Snowflake

In [57]:
success, nchunks, nrows, _ = write_pandas(conn, df, 'CA_LEGISLATIVE_DISTRICTS_TMP')
print(success, nchunks, nrows)

True 1 80


### Convert Polygon Coordinates to Geography Data Type and Save a Permanent Table with District Boundaries

In [60]:
conn.cursor().execute("CREATE OR REPLACE TABLE CA_LEGISLATIVE_DISTRICTS \
                       AS SELECT ID, AREA, DISTRICT, POPULATION,\
                       TO_GEOGRAPHY(POLYGON_COORDINATES) as DISTRICT_BOUNDARY \
                       from CA_LEGISLATIVE_DISTRICTS_TMP").fetchone()

('Table CA_LEGISLATIVE_DISTRICTS successfully created.',)

### Sample Spatial Join Query

In [61]:
conn.cursor().execute( "SELECT LD.DISTRICT, LD.POPULATION, COUNT(*) TOWER_CNT, \
                        ROUND((TOWER_CNT/POPULATION)*1000,0) TOWERS_PER_1000 \
                        from  GEO.TELCO.CELL_TOWERS_US CT, \
                        GEO.GEO_BOUNDARIES.CA_LEGISLATIVE_DISTRICTS LD \
                        Where ST_CONTAINS(LD.DISTRICT_BOUNDARY, CT.GEO) \
                        GROUP BY 1,2 \
                        ORDER BY 4 DESC;").fetchall()

[('34', 518705, 7267, 14),
 ('1', 506402, 7122, 14),
 ('9', 470020, 6296, 13),
 ('3', 496832, 5567, 11),
 ('32', 515015, 5083, 10),
 ('75', 471193, 4843, 10),
 ('36', 469902, 4579, 10),
 ('8', 501653, 5098, 10),
 ('27', 512647, 4690, 9),
 ('4', 475880, 4396, 9),
 ('30', 474319, 4369, 9),
 ('33', 505368, 4181, 8),
 ('2', 504804, 4223, 8),
 ('47', 518651, 4201, 8),
 ('12', 482778, 4023, 8),
 ('5', 498465, 3591, 7),
 ('22', 471588, 3235, 7),
 ('11', 514921, 3806, 7),
 ('42', 517503, 3420, 7),
 ('29', 495410, 3409, 7),
 ('35', 482837, 3032, 6),
 ('37', 474067, 2784, 6),
 ('23', 474000, 3057, 6),
 ('16', 516216, 3098, 6),
 ('39', 484755, 2471, 5),
 ('77', 508556, 2709, 5),
 ('71', 494346, 2504, 5),
 ('59', 470576, 2135, 5),
 ('31', 498044, 2487, 5),
 ('7', 491703, 2363, 5),
 ('76', 471670, 2437, 5),
 ('74', 512202, 2536, 5),
 ('41', 485018, 2417, 5),
 ('63', 478438, 2558, 5),
 ('60', 469942, 2424, 5),
 ('38', 485654, 2374, 5),
 ('13', 489925, 2458, 5),
 ('28', 473114, 2457, 5),
 ('15', 5122